# Analyzing Neural Time Series Data

#### Data extraction from MATLAB matrix as well as some functions written by Andrew J. Graves on 12/03/19

### Get the data

In [14]:
# Import modules
from scipy import io
import numpy as np
import mne
# My version of MNE is 0.19.2- AJG
# print(mne.__version__)

# Load Sample MATLAB/EEGLAB object from current directory
EEG = io.loadmat('sampleEEGdata.mat')['EEG']

# Get the data and reshape data from 3D (chan x time x trial) to 2D (chan x (time x trial))
eeg_data = EEG['data'].item()
two_d = eeg_data.reshape(eeg_data.shape[0], (eeg_data.shape[1]*eeg_data.shape[2]))

# Extract sampling rate, channel names, channel types, and time
samp_rate = EEG['srate'].item()
chan_names = sum(np.concatenate(EEG['chanlocs'].item()['labels'].tolist()).tolist(), [])
chan_types = ['eeg'] * len(chan_names)
eeg_time = EEG['times'].item()[0]

# Create an MNE info object and combine 2D data with info for an MNE raw object
mont = mne.channels.make_standard_montage('biosemi64')
info = mne.create_info(ch_names=chan_names, sfreq=samp_rate, ch_types=chan_types, montage=mont)
raw = mne.io.RawArray(two_d, info, verbose=False)

# Extract various features useful for plots
epochs = EEG['epoch'].item()[0]
chan_theta = EEG['chanlocs'].item()['theta']
chan_radius = EEG['chanlocs'].item()['radius'].tolist()

0.19.2


The following cells are an attempt to "modularize" some of Cohen's codes, by compacting them into re-useable functions. These instantiations of these pre-processing steps are directly translated from Cohen's code, and HAVE NOT been rigorously tested. Despite this, they may be useful. The functions also help me reduce error if I need to perform the same computation multiple times.

#### Chapter 9

In [45]:
# ERP
def get_erp(three_d, chan_name):
    """Returns an event-related potential at a specific channel
    
    Parameters
    ----------
    three_d: numpy array
        A 3-dimensional EEG data structure (channel x time x trial)
    
    chan_name: str
        The name of the channel to analyze
    
    Returns
    -------
    erp: An average voltage averaged across every trial at a specific channel
    """
    chan_index = [index for index, item in enumerate(chan_names) if item == which_chan]
    erp = np.squeeze(np.mean(three_d[list(chan_index), :, :], 2))
    return(erp)

# Low-pass filter for ERPs
def low_pass_erp(erp, filter_cutoff, trans_width, nyq):
    """Returns a low-pass filtered event-related potential
    
    Parameters
    ----------
    erp: numpy array
        A 1-dimensional event-related potential data structure
    
    filter_cutoff: int or float
        The threshold value for high frequencies to filter out
        
    trans_width: int or float
        The transition width of the filter
    
    nyq: int or float
        The nyquist frequency
    
    Returns
    -------
    low_pass: A low-pass filtered event-related potential
    """
    f_freqs = np.array([0, filter_cutoff, filter_cutoff * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([1, 1, 0, 0])
    filter_weights = signal.firls(101, f_freqs, ideal_resp)
    low_pass = signal.filtfilt(filter_weights, 1, erp)
    return(low_pass)

# Band-pass filter for ERPs
def band_pass_erp(erp, filter_low, filter_high, trans_width, nyq):
    """Returns a band-pass filtered event-related potential
    
    Parameters
    ----------
    erp: numpy array
        A 1-dimensional event-related potential data structure
    
    filter_low: int or float
        The threshold value for low frequencies to filter out
        
    filter_high: int or float
        The threshold value for high frequencies to filter out
        
    trans_width: int or float
        The transition width of the filter
    
    nyq: int or float
        The nyquist frequency
    
    Returns
    -------
    band_pass: A band-pass filtered event-related potential
    """
    f_freqs = np.array([0, filter_low * (1 - trans_width), filter_low, 
                        filter_high, filter_high * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([0, 0, 1, 1, 0, 0])
    filter_weights = signal.firls(round(3 * (samp_rate.item() / filter_low) + 1), f_freqs, ideal_resp)
    band_pass = signal.filtfilt(filter_weights, 1, erp)
    return(band_pass)

#### Chapter 11

In [ ]:
# Generate a sine wave
def get_sine_wave(frequency, amplitude, time_vec, phase=0):
    """Returns a simulated sine wave
    
    Parameters
    ----------
    frequency: int or float
        Frequency of the sine wave
    
    amplitude: int or float
        Amplitude of the sine wave
        
    time_vec: numpy array
        A 1-dimensional array for the time axis of the sine wave
        
    phase: int or float
        The phase angle offset of the sine wave. Defaults to 0
    
    Returns
    -------
    sine-wave: The simulated sine wave
    """
    sine_wave = amplitude * np.sin(2 * np.pi * frequency * time_vec + phase)
    return sine_wave

# Compute power with numpy fourier transform
def compute_power(signal, srate):
    """Compute power with numpy fourier transform. Note that Cohen multiplied the series by 2 in the code 
       rather than squaring it for some reason. AFAIK squaring is appropriate, which is why the magnitudes 
       of my frequencies are larger
    
    Parameters
    ----------
    signal: numpy array
        A 1-dimensional signal array
    
    srate: int or float
        The sampling rate of the signal
    
    Returns
    -------
    power: The power at particular frequencies
    frequencies: The frequency index for each power entry
    """
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    power = np.abs(fft_sig[0:len(frequencies)]) ** 2
    return power, frequencies

# Compute phase with numpy fourier transform
def compute_phase(signal, srate):
    """Compute phase with numpy fourier transform.
    
    Parameters
    ----------
    signal: numpy array
        A 1-dimensional signal array
    
    srate: int or float
        The sampling rate of the signal
    
    Returns
    -------
    phase: The phase at particular frequencies
    frequencies: The frequency index for each phase entry
    """
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    phase = np.angle(fft_sig[0:len(frequencies)], deg=False)
    return phase, frequencies

#### Might be useful later...

In [ ]:
# Convert from Polar to Cartesian coordinates
def pol_to_cart(phi, rho):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return x, y